# Transformer Classification Experiment

In [1]:
__author__ = "JUN WEI WANG"
__email__ = "wjw_03@outlook.com"

In [2]:
import torch
import numpy as np
import os
from os import path

import matplotlib.pyplot as plt
# from IPython.display import set_matplotlib_formats
# %matplotlib inline
# set_matplotlib_formats('svg')

# Check for CUDA!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently training on: {device}")

print("Your current working directory", os.getcwd())

Currently training on: cuda
Your current working directory C:\Users\wjw_0\dsp-project


In [3]:
import src.generator.QAM as QAM

# MODE = "TRAINING"
MODE = "INFERENCE"
print(f"MODE: {MODE}")

DATASET_SOURCE = "394" # 实验数据序号 或者 SYNTH
# DATASET_SOURCE = "SYNTH"

SEED = 1
CONSTELLATION = "QAM"
QAM_ORDER = 64
NUMBEROFSYMBOLS = 51200
TAPS = 35

cons = QAM.read_constellation_file(QAM_ORDER, CONSTELLATION)

MODE: INFERENCE


In [4]:
import torchvision
import torch.nn as nn
from tqdm import notebook


In [5]:
# Data processing

import json

# Input JSON data
json_data = """
{
  "data": [
    {
      "ID": 357,
      "I": 60,
      "VPP": 300,
      "best": 0.042237062
    },
    {
      "ID": 359,
      "I": 60,
      "VPP": 350,
      "best": 0.0047104652
    },
    {
      "ID": 361,
      "I": 60,
      "VPP": 400,
      "best": 0.0076074503
    },
    {
      "ID": 363,
      "I": 60,
      "VPP": 450,
      "best": 0.009638712
    },
    {
      "ID": 367,
      "I": 60,
      "VPP": 500,
      "best": 0.0052374873
    },
    {
      "ID": 399,
      "I": 60,
      "VPP": 550,
      "best": 0.024265933
    },
    {
      "ID": 401,
      "I": 60,
      "VPP": 600,
      "best": 0.039526662
    },
    {
      "ID": 352,
      "I": 70,
      "VPP": 300,
      "best": 0.0042554584
    },
    {
      "ID": 350,
      "I": 70,
      "VPP": 350,
      "best": 0.0033094373
    },
    {
      "ID": 343,
      "I": 70,
      "VPP": 400,
      "best": 0.0021048152
    },
    {
      "ID": 345,
      "I": 70,
      "VPP": 450,
      "best": 0.0028151494
    },
    {
      "ID": 347,
      "I": 70,
      "VPP": 500,
      "best": 0.0044584111
    },
    {
      "ID": 355,
      "I": 70,
      "VPP": 550,
      "best": 0.00954204722
    },
    {
      "ID": 397,
      "I": 70,
      "VPP": 600,
      "best": 0.012746735
    },
    {
      "ID": 326,
      "I": 80,
      "VPP": 450,
      "best": 0.0022881273
    },
    {
      "ID": 329,
      "I": 80,
      "VPP": 500,
      "best": 0.0026318374
    },
    {
      "ID": 332,
      "I": 80,
      "VPP": 550,
      "best": 0.0030213755
    },
    {
      "ID": 334,
      "I": 80,
      "VPP": 600,
      "best": 0.0051458313
    },
    {
      "ID": 336,
      "I": 80,
      "VPP": 650,
      "best": 0.0073815837
    },
    {
      "ID": 339,
      "I": 80,
      "VPP": 700,
      "best": 0.010877606
    },
    {
      "ID": 341,
      "I": 80,
      "VPP": 750,
      "best": 0.017869652
    },
    {
      "ID": 394,
      "I": 90,
      "VPP": 300,
      "best": 0.017869652
    },
    {
      "ID": 386,
      "I": 90,
      "VPP": 350,
      "best": 0.0062129693
    },
    {
      "ID": 384,
      "I": 90,
      "VPP": 400,
      "best": 0.0060983993
    },
    {
      "ID": 309,
      "I": 90,
      "VPP": 450,
      "best": 0.0028020557
    },
    {
      "ID": 311,
      "I": 90,
      "VPP": 500,
      "best": 0.0051752922
    },
    {
      "ID": 314,
      "I": 90,
      "VPP": 550,
      "best": 0.0040164981
    },
    {
      "ID": 316,
      "I": 90,
      "VPP": 600,
      "best": 0.004579528
    },
    {
      "ID": 318,
      "I": 90,
      "VPP": 650,
      "best": 0.0060558447
    },
    {
      "ID": 321,
      "I": 90,
      "VPP": 700,
      "best": 0.0063700939
    },
    {
      "ID": 323,
      "I": 90,
      "VPP": 750,
      "best": 0.012262267
    },
    {
      "ID": 369,
      "I": 100,
      "VPP": 300,
      "best": 0.01826901
    },
    {
      "ID": 371,
      "I": 100,
      "VPP": 350,
      "best": 0.0095387738
    },
    {
      "ID": 373,
      "I": 100,
      "VPP": 400,
      "best": 0.0071524436
    },
    {
      "ID": 375,
      "I": 100,
      "VPP": 450,
      "best": 0.0056663066
    },
    {
      "ID": 377,
      "I": 100,
      "VPP": 500,
      "best": 0.0070280533
    },
    {
      "ID": 379,
      "I": 100,
      "VPP": 550,
      "best": 0.0069625847
    },
    {
      "ID": 381,
      "I": 100,
      "VPP": 600,
      "best": 0.0069822253
    }
  ]
}
"""

# Load the JSON string into a Python dictionary
data = json.loads(json_data)

# Initialize an empty dictionary to store the mapping
iv_to_id_map = {}

# Process each entry in the "data" list
for entry in data['data']:
    i = entry['I']
    vpp = entry['VPP']
    id_val = entry['ID']
    iv_to_id_map[(i, vpp)] = id_val

# Output the organized dictionary
print(iv_to_id_map)

# Collect all unique (I, VPP) combinations
unique_combinations = sorted(set((entry['I'], entry['VPP']) for entry in data['data']))

# Create a mapping from combination to index
combination_to_index = {combo: idx for idx, combo in enumerate(unique_combinations)}

# Generate one-hot-encoded vectors for each entry in the data
one_hot_vectors = {}
for entry in data['data']:
    combo = (entry['I'], entry['VPP'])
    vector = np.zeros(len(unique_combinations))
    vector[combination_to_index[combo]] = 1
    one_hot_vectors[combo] = vector

# Output the one-hot vectors for verification
for combo, vector in one_hot_vectors.items():
    print(f"{combo}: {vector.tolist()}")
    


{(60, 300): 357, (60, 350): 359, (60, 400): 361, (60, 450): 363, (60, 500): 367, (60, 550): 399, (60, 600): 401, (70, 300): 352, (70, 350): 350, (70, 400): 343, (70, 450): 345, (70, 500): 347, (70, 550): 355, (70, 600): 397, (80, 450): 326, (80, 500): 329, (80, 550): 332, (80, 600): 334, (80, 650): 336, (80, 700): 339, (80, 750): 341, (90, 300): 394, (90, 350): 386, (90, 400): 384, (90, 450): 309, (90, 500): 311, (90, 550): 314, (90, 600): 316, (90, 650): 318, (90, 700): 321, (90, 750): 323, (100, 300): 369, (100, 350): 371, (100, 400): 373, (100, 450): 375, (100, 500): 377, (100, 550): 379, (100, 600): 381}
(60, 300): [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(60, 350): [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [ ]:
from typing import List, Tuple, Any
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from os import path

# Local
from src.data.processing import parse_str
from src.data.file_handler import read_file
class CustomDataset(Dataset):
    def __init__(
            self,
            folder_path: str = None,
            data_files:  List[str] = None,
            labels:      List[Any] = None,
            label_mapping: dict[Any, int] = None,
            classes=None,
            num_samples: int = 512,
          ):

        if not folder_path or not data_files or not labels or label_mapping is None:
            self.dataset = None
            self.labels = None
            self.size = 0
            self.num_classes = 0
            return

        all_datasets = []
        all_labels = []

        for data_file, label in zip(data_files, labels):
            data_path = path.join(folder_path, data_file)
            dataset = parse_str(read_file(data_path))
            sequences, seq_labels = self.split_sequence(
                dataset,
                label_mapping[label],  # Encode label as integer index
                num_samples
            )
            all_datasets.append(sequences)
            all_labels.append(seq_labels)

        # Concatenate all datasets and labels
        self.dataset = np.concatenate(all_datasets, axis=0)
        self.labels = np.concatenate(all_labels, axis=0)

        self.classes = classes

        # Convert to tensor
        self.dataset = torch.tensor(self.dataset, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)  # Use long for class indices
        # Rearrange dimensions to (features, sequence_length) ➔ (2, 512)
        # print(self.dataset.shape)

        self.dataset = self.dataset.squeeze().permute(1, 0)  # Swaps the last two dimensions
        # Store the number of unique classes
        self.num_classes = len(label_mapping)

        print(f"Number of classes: {self.num_classes}")
        print("Sample data:", self.dataset[:2])
        print("Sample labels:", self.labels[:2])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx], self.labels[idx]

    @staticmethod
    def split_sequence(
            dataset: List[List[float]], label: int, win_len: int
        ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Splits the dataset into sequences of length `win_len` and assigns the provided label.

        Args:
            dataset (List[List[float]]): The input dataset as a list of lists.
            label (int): The integer label to assign to each sequence.
            win_len (int): The length of the sliding window.

        Returns:
            Tuple[np.ndarray, np.ndarray]: A tuple containing:
                - `new_dataset` (np.ndarray): Sequences split from the dataset.
                - `new_labels` (np.ndarray): Labels for each sequence.
        """
        if len(dataset) < win_len:
            raise ValueError("Dataset length must be greater than or equal to the window length.")

        new_dataset = []
        new_labels = []

        # Sliding window implementation
        for j in range(len(dataset) - win_len + 1):
            start_index = j
            end_index = start_index + win_len
            new_dataset.append(dataset[start_index:end_index])
            new_labels.append(label)

        return np.array(new_dataset), np.array(new_labels)

# Prepare the list of data files and their corresponding labels
data_files = []
labels = []

i = 0
for (label, set_num) in iv_to_id_map.items():
    data_file = f"OSC_sync_{set_num}.txt"  # Assuming file names are based on set_num
    print(data_file)
    data_files.append(data_file)
    labels.append(label)  # Labels are tuples like (I, VPP)
    i = i + 1
    if i == 1:
        break

# Create a mapping from labels to integer indices
unique_labels = sorted(set(labels))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
num_classes = len(unique_labels)

print(unique_labels)

# Create the dataset
dataset = CustomDataset(
    folder_path="./dataset",
    data_files=data_files,
    labels=labels,
    label_mapping=label_mapping,
    classes=unique_labels,
    num_samples=512
)

TRAIN_RATIO = 0.6
BATCH_SIZE = 512

from torch.utils.data import Subset

class CustomSubset(Subset):
    def __init__(self, dataset, indices):
        super().__init__(dataset, indices)
        # Copy custom attributes
        self.num_classes = dataset.num_classes
        self.classes = dataset.classes
        # Add any other custom variables you need

# Modify your splitting code
def custom_random_split(dataset, lengths):
    """
    Splits the dataset into non-overlapping new datasets of given lengths.
    Returns a list of CustomSubset instances.
    """
    from torch.utils.data import random_split
    subsets = random_split(dataset, lengths)
    return [CustomSubset(subset.dataset, subset.indices) for subset in subsets]

train_size = int(TRAIN_RATIO * len(dataset))
print(f"Traning ratio (train:valid): {TRAIN_RATIO}:{1 - TRAIN_RATIO}")
print(f"Training size: {train_size} | Validation size: {len(dataset) - train_size}")
val_size = len(dataset) - train_size
train_dataset, val_dataset = custom_random_split(
    dataset, [train_size, val_size]
)
print(type(train_dataset))
# Setup the training and validation data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)



OSC_sync_357.txt
[(60, 300)]


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 1 is not equal to len(dims) = 2

In [7]:
from src.nn.TRANSFORMER import Hybrid

model = Hybrid(input_samples=512, n_classes=num_classes,debug=False)
model.to(device)
model = nn.DataParallel(model, device_ids=[0])


In [8]:

conf_matrix = np.zeros((len(train_dataset.classes),len(train_dataset.classes)))
conf_matrix_2 = np.zeros((len(val_dataset.classes),len(val_dataset.classes)))
epoch_nums = 16
best_accuracy = -1000

criterion_classifier = nn.CrossEntropyLoss()
criterion_recon = nn.MSELoss()
optimizer =  torch.optim.Adam(model.parameters(), lr=.001)

In [9]:

alpha = 1
beta = 1

for epoch in notebook.tqdm(range(epoch_nums),desc='Epoch'):
    model.train()
    for item in notebook.tqdm(train_loader,desc='Training',leave=False):
        # print(item[1])
        # RXinputs = item['data'].to(device)
        # TXinputs = item['data_Tx'].to(device)
        # labels = item['label'].to(device)
        RXinputs = item[0].to(device)
        labels = item[1].to(device)

        print(RXinputs.shape)

        optimizer.zero_grad()
        outputs = model(RXinputs)
        
        _, predicted = torch.max(outputs.data, 1)
        
        loss_classify = alpha*criterion_classifier(outputs,labels)
        # loss_recon = beta*(criterion_recon(recon,TXinputs))
        # loss = loss_classify + loss_recon
        loss = loss_classify
        
        loss.backward()
        optimizer.step()
        
        if epoch == epoch_nums-1:
            for i in range(RXinputs.shape[0]):
                label = labels[i]
                pred = predicted[i]
                conf_matrix[label,pred]+=1
        
    correct = 0    
    model.eval()    
    for item in notebook.tqdm(val_loader,desc='Validation',leave=False):
        RXinputs = item['data'].to(device)
        TXinputs = item['data_Tx'].to(device)
        labels = item['label'].to(device)
        optimizer.zero_grad()
        outputs,recon = model(RXinputs)
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().cpu().data.numpy()
        
        if epoch == epoch_nums-1:
            for i in range(RXinputs.shape[0]):
                label = labels[i]
                pred = predicted[i]
                conf_matrix_2[label,pred]+=1
        
    accuracy = correct/float(len(val_loader)*BATCH_SIZE)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        #torch.save(model.state_dict(), "Trained_Models/classification.pt")    

    if epoch == epoch_nums-1: 
        for i in range(len(conf_matrix_2)):
            conf_matrix_2[i]=conf_matrix_2[i]/conf_matrix_2[i].sum() 
            
        for i in range(len(conf_matrix)):
            conf_matrix[i]=conf_matrix[i]/conf_matrix[i].sum() 

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

Training:   0%|          | 0/240 [00:00<?, ?it/s]

torch.Size([512, 512, 1])


RuntimeError: Given groups=1, weight of size [128, 2, 1], expected input[512, 512, 1] to have 2 channels, but got 512 channels instead